In [2]:
#!/usr/bin/env python



In [2]:
import pandas as pd
import numpy as np

#import pandas.io.sql as pd_sql
from sodapy import Socrata
#import requests 
#import sqlite3 as sq 

from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
# import collect_data


In [202]:

client = Socrata('data.cityofchicago.org','gz7GKwbVG8X0YFDvbgQXlOnrx')
offset_temp = 0
loop = True  
while loop:
    results = client.get('9kgb-ykyt', select="trip_id,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude",
                         limit=10000, offset=offset_temp)
    trips_df_temp = pd.DataFrame.from_records(results)
    # This loop stops when the next block of dataframe is empty
    if trips_df_temp.empty == True:
        loop = False
            
    trips_df_temp.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True) 

            
            
            
    # Insert dataframe into TaxiTrips2.db
    engine = create_engine('sqlite:///TaxiTrips.db')
    trips_df_temp.to_sql("taxi_2021", con=engine, if_exists="append", index=False)   
    offset_temp+=10000

     # Shows the percentage of data 
    if offset_temp % 100000 == 0:
        print(offset_temp/2576796*100,"%")
    else:
        continue
            

3.8807883899229894 %
7.761576779845979 %
11.642365169768969 %
15.523153559691957 %
19.403941949614946 %
23.284730339537937 %
27.16551872946093 %
31.046307119383915 %
34.9270955093069 %
38.80788389922989 %
42.68867228915288 %
46.569460679075874 %
50.450249068998865 %
54.33103745892186 %
58.211825848844846 %
62.09261423876783 %
65.97340262869082 %
69.8541910186138 %
73.7349794085368 %
77.61576779845979 %
81.49655618838278 %
85.37734457830577 %
89.25813296822876 %
93.13892135815175 %
97.01970974807475 %


In [130]:

convert_dict = {'trip_seconds': 'float32', 
                'trip_miles': 'float32', 
                'dropoff_community_area': 'float32',
                'fare': 'float32', 
                'trip_total': 'float32',
                'dropoff_centroid_latitude': 'float32',
                'dropoff_centroid_longitude': 'float32',
                'pickup_community_area': 'float32',
                'pickup_centroid_latitude': 'float32',
                'pickup_centroid_longitude': 'float32'}
  


In [322]:
engine = create_engine('sqlite:///TaxiTrips.db')
conn = engine.connect().execution_options(stream_results=True)
    
chunk_list = []
for chunk_dataframe in pd.read_sql(
            'SELECT * FROM taxi_2021 WHERE dropoff_community_area != "None" AND fare != "None" AND trip_total !="None" AND payment_type != "None" AND company != "None" AND dropoff_centroid_latitude != "None" AND dropoff_centroid_longitude != "None" AND pickup_community_area != "None" AND pickup_centroid_latitude != "None" AND pickup_centroid_longitude != "None" AND trip_seconds!=0 and trip_miles !=0' , conn, chunksize=1000):
    
    
    #print(f"Got dataframe w/{len(chunk_dataframe)} rows")
        
        
   
    chunk_list.append(chunk_dataframe)

df_concat = pd.concat(chunk_list)    

In [323]:
df =df_concat.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1843982 entries, 0 to 981
Data columns (total 14 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   trip_id                     object
 1   trip_start_timestamp        object
 2   trip_seconds                object
 3   trip_miles                  object
 4   dropoff_community_area      object
 5   fare                        object
 6   trip_total                  object
 7   payment_type                object
 8   company                     object
 9   dropoff_centroid_latitude   object
 10  dropoff_centroid_longitude  object
 11  pickup_community_area       object
 12  pickup_centroid_latitude    object
 13  pickup_centroid_longitude   object
dtypes: object(14)
memory usage: 211.0+ MB


In [ ]:
def reorg_col(df):
    df = df.astype(convert_dict)
    df['trip_start_timestamp'] = pd.to_datetime(df["trip_start_timestamp"], format="%Y-%m-%dT%H:%M:%S.%f")
    df['trip_date'] =(df['trip_start_timestamp']).dt.date
    df['trip_day'] = (df['trip_start_timestamp']).dt.dayofweek
    df['trip_start_time'] =df['trip_start_timestamp'].dt.time
    df['trip_month'] = df['trip_start_timestamp'].dt.month_name
    df['trip_day_n']=df['trip_start_timestamp'].dt.day_name()
    df['trip_timezone'] = np.digitize(df['trip_start_timestamp'].dt.hour, [0,5,10,16,20])
    df=df.assign(timezone=pd.cut(df_full['trip_start_timestamp'].dt.hour,
                            [0,6,12,18,23],
                            labels=['Night','Morning','Afternoon','Evening'],
                            include_lowest=True))
    return df


In [ ]:
df= df.reorg_col(df)
with open("trips_2021.pkl", "wb") as f:
    pkl.dump(df,f)